In [1]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#installing libraries
!pip install faiss-gpu
!pip install datasets
!pip install sentence-transformers
!pip install faiss
!pip install wget
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.3 MB/s eta 0:

In [3]:
#importing libraries
import faiss
from datasets import load_dataset
import json
import jsonlines
import faiss
import numpy as np
import wget
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import numpy as np

In [4]:
#loading spider dataset
data = load_dataset("spider")

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

Dataset spider downloaded and prepared to /root/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
%cd /content/drive/MyDrive/1_spring_23/523/Project/rasat

/content/drive/MyDrive/1_spring_23/523/Project/rasat


In [6]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.6/851.6 kB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.0/432.0 kB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 64.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━

In [7]:
# split train dataset into train and retrieval augmentation dataset
def create_ra_dataset(data):
    
    data = data['train']
    data = data.train_test_split(test_size=0.1)
    return data['train'], data['test']

In [35]:
def create_index(ra_dataset, tokenizer, model):
    # Tokenize the text
    text_embeddings = tokenizer(ra_dataset['question'], return_tensors='pt', padding=True, truncation=True)

    # Get embeddings from the model
    with torch.no_grad():
        model_output = model(**text_embeddings)

    # Extract the embeddings (assuming the embeddings are stored in the last_hidden_state)
    embeddings = model_output.last_hidden_state.mean(dim=1).numpy()

    # Create a Faiss index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    return index

In [36]:
from transformers import AutoTokenizer, AutoModel

#model 1 - ret aug
# Download and load the Spider dataset
with open('/content/drive/MyDrive/1_spring_23/523/Project/rasat/dataset_files/ori_dataset/spider/train_spider.json', 'r') as f:
    spider_data = json.load(f)

# Extract text data and corresponding SQL queries
texts = [item['question'] for item in spider_data]
sql_queries = [item['query'] for item in spider_data]

# Preprocess the data by creating sentence pairs
sentence_pairs = list(zip(texts[:-1], texts[1:]))

#model 1 tokenizer
tokenizer_1 = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-distilroberta-base-v1')

# Initialize Sentence-BERT
model_1 = AutoModel.from_pretrained('sentence-transformers/paraphrase-distilroberta-base-v1')

#tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-distilroberta-base-v1')
#model = AutoModel.from_pretrained('sentence-transformers/paraphrase-distilroberta-base-v1')

# Create InputExamples for the dataset
examples = [InputExample(texts=[pair[0], pair[1]], label=1.0) for pair in sentence_pairs]

# Create a DataLoader for the dataset
train_dataloader = DataLoader(examples, shuffle=True, batch_size=16)

# Define a loss function
train_loss = losses.CosineSimilarityLoss(model_1)

# Fine-tune Sentence-BERT
#model_1.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

In [37]:
train_dataset, ra_dataset = create_ra_dataset(data)

In [39]:
index = create_index(ra_dataset, tokenizer_1, model_1)

In [42]:
def retrieval_augmentation(train_dataset, index, ra_dataset, tokenizer, model, k=1):
    
    def augment_question(example):
        X = example['question']
        # Tokenize the text
        text_embeddings = tokenizer(X, return_tensors='pt', padding=True, truncation=True)

        # Get embeddings from the model
        with torch.no_grad():
            model_output = model(**text_embeddings)

        # Extract the embeddings (assuming the embeddings are stored in the last_hidden_state)
        new_text_embedding = model_output.last_hidden_state.mean(dim=1).numpy()
        
        # Find the most similar text in the dataset
        _, indices = index.search(new_text_embedding, k)
        similar_text = ra_dataset['question'][indices[0][0]]
        
        # Find the corresponding SQL query for the most similar text
        query = ra_dataset['query'][indices[0][0]]
        
        # Append the original text and its corresponding SQL query with separator token
        query = query.replace("\n", " ")
        query = query.replace("\t", " ")
        augmented_question = X + "[SEP]" + query
        
        example['question'] = augmented_question
        return example

    # Update the train_dataset using the map function
    augmented_dataset = train_dataset.map(augment_question)
    
    return augmented_dataset

In [43]:
new_data = retrieval_augmentation(train_dataset, index, ra_dataset, tokenizer_1, model_1)

Map:   0%|          | 0/6300 [00:00<?, ? examples/s]

In [44]:
from torch.utils.data import Dataset

In [45]:
#v2
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=None):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_text = self.data[idx]['question']
        target_text = self.data[idx]['query']

        encoded_input = self.tokenizer(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')
        encoded_labels = self.tokenizer(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt')

        return {
            'input_ids': encoded_input['input_ids'].squeeze(),
            'attention_mask': encoded_input['attention_mask'].squeeze(),
            'labels': encoded_labels['input_ids'].squeeze(),
        }

In [46]:
#tokenizer 2
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer_2 = AutoTokenizer.from_pretrained("Jiexing/spider_relation_t5_3b-2624")

In [47]:
train_dataset = CustomDataset(new_data, tokenizer_2, max_length=128)

In [48]:
#model 2 - for validation 
from transformers import T5Config
from seq2seq.model.t5_relation_model import T5ForConditionalGeneration
from seq2seq.model.t5_relation_config import RASATConfig  # Import the modified model class

# Load the pre-trained weights into the modified model class
config = RASATConfig.from_pretrained("Jiexing/spider_relation_t5_3b-2624")
model_2 = T5ForConditionalGeneration.from_pretrained("Jiexing/spider_relation_t5_3b-2624", config=config)

In [50]:
new_data_val = retrieval_augmentation(data['validation'], index, ra_dataset, tokenizer_1, model_1)

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [58]:
new_eval = CustomDataset(new_data_val, tokenizer_2, max_length=128)

In [59]:
from transformers import Trainer, TrainingArguments

# Define your training data and other necessary variables

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
)

In [60]:
trainer = Trainer(
    model=model_2,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=new_eval,
    #compute_metrics = compute_metrics  # Add the validation dataset here
    # Other optional arguments for evaluation/validation data, data collator, etc.
)

In [61]:
import gc
gc.collect()
import torch
torch.cuda.empty_cache()

In [62]:
evaluation_results = trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:862: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:816: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [63]:
evaluation_results

{'eval_loss': 44.805416107177734,
 'eval_runtime': 46.707,
 'eval_samples_per_second': 22.138,
 'eval_steps_per_second': 2.783}